# Claro loop + fit

In [1]:
from claro_fit import *

In [2]:
a = fileinfo('C:\\Users\\MARTINA\\Desktop\\secondolotto_1\\')

The default subfolders' paths are *Station_1__*\Station_1__??_Summary\Chip_???\S_curve.
The default file names are Ch_*_offset_*_Chip_*.txt.
To confirm, press Enter. Press any other key to change the paths.
 


ValueError: Bad data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_218\S_curve\Ch_2_offset_0_Chip_218.txt. First line: Troppi datia. Going on...
ValueError: Bad data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_219\S_curve\Ch_2_offset_0_Chip_219.txt. First line: Troppi dati
ValueError: Bad data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_241\S_curve\Ch_1_offset_0_Chip_241.txt. First line: Troppi datia. Going on...
ValueError: Bad data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_247\S_curve\Ch_0_offset_0_Chip_247.txt. First line: Troppi dati
ValueError: Bad data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_251\S_curve\Ch_0_offset_0_Chip_251.txt. First line: Troppi datia. Going on...
ValueError: Bad data in: C:\Users\MARTINA\Desktop\secondolotto_1\Station_1__11\Station_1__11_Summary\Chip_25

In [3]:
fileinfo_find(a, chip='002', ch='1')

'C:\\Users\\MARTINA\\Desktop\\secondolotto_1\\Station_1__11\\Station_1__11_Summary\\Chip_002\\S_curve\\Ch_1_offset_0_Chip_002.txt'

In [12]:
def plot_fit(x, y, metafit, fileinfo, npoints=1000,
             interactive=False,
             show_scatter=True, show_fit=True, show_transition=True,
             save=False, save_path='.\plot', save_format='pdf', show=True,
             **kwargs):
    """
    [Function documentation]
    """
    

    title = f"Fit CLARO: station {fileinfo['station']}, chip {fileinfo['chip']}, ch {fileinfo['ch']}"

    try:
        if metafit['code']=='bad':
            raise ValueError("Error: the provided fit has invalid parameters. "
                             +f"Plot {title} will not be plotted.\r")
            
        font = {
            'fontsize': plt.rcParams['axes.titlesize'],
            'fontweight' : plt.rcParams['axes.titleweight'],
            'verticalalignment': 'baseline',
            # 'horizontalalignment': plt.loc,
        }
        
        plt.figure(figsize=(5,5), tight_layout=True)
        plt.title(title, fontdict=font)
    
        # x,y data computed as a grid of npoints and using parameters of the fit
        fit_params = metafit['params']
        fit_errs = metafit['errors']
        
        xfit = np.linspace(x.min(), x.max(), npoints)
        yfit= func(xfit, *fit_params)
        
        # Show scatterplot of data (default = True)
        if show_scatter == True:
            plt.scatter(x, y, label='Data points', zorder=2)
            plt.annotate(f"From data file\nTransition = {fileinfo['transition']:.2f}\n"
                         +f"Width = {fileinfo['width']:.2f}",
                         xy=(0.025, .975), xycoords='axes fraction',
                         verticalalignment='top', color='gray', alpha=0.8)
            plt.xlabel('x [arb units]', fontdict=font)
            plt.ylabel('y [arb units]', fontdict=font)
            # plt.legend(loc='lower right', fontsize = 8)
        
                             
        # Show plot of fit (default = True)
        if show_fit == True: 
            plt.plot(xfit, yfit, label='erf fit', zorder=1, color='r', alpha=0.8)
            plt.annotate(f"Fit parameters\nAmplitude = ({fit_params[0]:.2f}"+r" $\pm$ "+f"{fit_errs[0]:.2f})\n"
                         +f"Transition = ({fit_params[1]:.2f}"+r" $\pm$ "+f"{fit_errs[1]:.2E})\n"
                         +f"Width = ({fit_params[2]*2:.2f}"+r" $\pm$ "+f"{fit_errs[2]*2:.2E})",
                         xy=(0.025, .3), xycoords='axes fraction', verticalalignment='top',
                         color='r', alpha=0.8,
                         bbox=dict(boxstyle="square", fc="white", ec="black", lw=1, alpha=0.8))
            
        # Show transition point
        if show_transition == True:
            xtrans = fit_params[1]
            ytrans = func(xtrans, *fit_params)
            plt.scatter(xtrans, ytrans, s=100, c='darkred', marker='*',
                        zorder=2, label='Transition point')
            plt.vlines(x=xtrans, ymin=y[0], ymax=y[-1], linestyles='dashed',
                        alpha=0.5, zorder=1)
            plt.hlines(y=ytrans, xmin=x[0], xmax=x[-1], linestyles='dashed',
                        alpha=0.5, zorder=1)
            plt.annotate(f"Transition = {fit_params[1]:.2f}\nWidth = {fit_params[2]*2:.2f}", 
                         xy=(0.025, .6), xycoords='axes fraction', verticalalignment='top',)

    except ValueError as err: 
        if interactive==True: print(err)
    
    if save == True:
        plt.savefig(save_path+'.'+save_format)
    
    if show == True:
        plt.show()
    else:
        plt.close()

In [13]:
def func(x, ampl, a, b):
    """Modified erf function.
    
    Amplitude ampl and shift on the vertical axis ampl/2,
    inflection point at a, normalization proportional to b.
    Deafult guesses are [AMPLITUDE, TRANSITION, AMPLITUDE/2].
    """
    
    return ampl/2*(1+sp.erf((x-a)/(b*np.sqrt(2))))

In [ ]:
tot = 0
counter = 0
for file in a.values(): # file is the sub-dict, access keys via file['key']
    tot += 1
    x,y,meta = read_data(file['path'])
    metafit = fit_erf(x,y,meta, interactive=False)
    plot_fit(x, y, metafit, fileinfo=file, show=False, show_fit=True, show_transition=True, save=False)

C:\Anaconda3\envs\oop\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Anaconda3\envs\oop\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Anaconda3\envs\oop\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\MARTINA\Desktop\OOPROJECT\pyproj\branch_claro\Def\claro_fit.py:193: RuntimeWarning: invalid value encountered in sqrt
  except ValueError as err:
C:\Users\MARTINA\Desktop\OOPROJECT\pyproj\branch_claro\Def\claro_fit.py:193: RuntimeWarning: invalid value encountered in sqrt
  except ValueError as err:
C:\Anaconda3\envs\oop\lib\site-packages\scipy\optimize\mi